In [20]:
import pandas as pd
import os
from glob import glob
import numpy as np
from pandas.api.types import CategoricalDtype

In [21]:
ex1 = pd.read_parquet('train/4.청구입금정보/2018_청구정보_전체.parquet')

In [22]:
member_df = pd.read_parquet('train/1.회원정보/2018_회원정보_전체.parquet')
member_seg = member_df[['ID', 'Segment']].drop_duplicates(subset='ID')
ex1 = ex1.merge(member_seg, on='ID', how='left')



In [23]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '대표결제일', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_포인트_월적립_B0M', '포인트_포인트_월적립_R3M', '포인트_적립포인트_R12M', '포인트_적립포인트_R3M', '포인트_이용포인트_R12M', '포인트_이용포인트_R3M', '포인트_잔여포인트_B0M', '마일_적립포인트_R12M', '마일_적립포인트_R3M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', '마일_잔여포인트_B0M', '할인건수_R3M', '할인금액_R3M', '할인건수_B0M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '선결제건수_R6M', '선결제건수_R3M', '연체건수_R6M', '연체건수_R3M', '혜택수혜금액_R3M', '포인트_마일리지_환산_B0M', '혜택수혜금액', 'Segment']

문자형 컬럼 이름:
['ID', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '할인건수_R3M', '할인건수_B0M', 'Segment']

숫자형 컬럼 이름:
['기준년월', '대표결제일', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M',

In [24]:
cols = ['대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '할인건수_R3M', '할인건수_B0M']
print(ex1[cols].head())

  대표결제방법코드 대표청구지고객주소구분코드 대표청구서수령지구분코드 청구서수령방법 할인건수_R3M 할인건수_B0M
0     자동이체           미확인    당사페이앱+이메일   문자메세지    1회 이상    1회 이상
1     자동이체           주거지           우편      우편    1회 이상    1회 이상
2     자동이체           미확인          이메일     이메일    1회 이상    1회 이상
3     자동이체           주거지           우편      우편    1회 이상    1회 이상
4     자동이체           주거지           우편      우편    1회 이상    1회 이상


In [25]:
print("할인건수_R3M unique values:")
print(ex1['할인건수_R3M'].unique())

print("\n할인건수_B0M unique values:")
print(ex1['할인건수_B0M'].unique())

할인건수_R3M unique values:
['1회 이상' '10회 이상' '20회 이상' '30회 이상' '40회 이상']

할인건수_B0M unique values:
['1회 이상' '10회 이상']


In [26]:
count_cols2 = ['할인건수_R3M', '할인건수_B0M']

for col in count_cols2:
    # 1) 문자열로 바꾼 뒤 “회” 제거
    ex1[col] = ex1[col].astype(str).str.replace('회 이상', '', regex=False)
    
    # 2) 숫자로 변환, 변환 불가(예: 'nan', 'None')는 NaN → fillna(-1) → 정수형
    ex1[col] = (
        pd.to_numeric(ex1[col], errors='coerce')
          .fillna(-1)
          .astype(int)
    )

In [27]:
le_cols = ['대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법']

for col in le_cols:
    codes, uniques = pd.factorize(ex1[col], sort=True)
    ex1[col] = codes


In [28]:
ex1.head()

,기준년월,ID,대표결제일,대표결제방법코드,대표청구지고객주소구분코드,대표청구서수령지구분코드,청구서수령방법,청구서발송여부_B0,청구서발송여부_R3M,청구서발송여부_R6M,...,상환개월수_결제일_R6M,상환개월수_결제일_R3M,선결제건수_R6M,선결제건수_R3M,연체건수_R6M,연체건수_R3M,혜택수혜금액_R3M,포인트_마일리지_환산_B0M,혜택수혜금액,Segment
0,201807,TRAIN_000000,27,0,0,2,2,1,1,1,...,5,3,0,0,1,0,3,0,0,D
1,201807,TRAIN_000001,13,0,1,5,4,1,1,1,...,6,3,0,0,0,0,0,0,0,E
2,201807,TRAIN_000002,1,0,0,6,5,1,1,1,...,6,3,0,0,0,0,121,0,50,C
3,201807,TRAIN_000003,5,0,1,5,4,1,1,1,...,5,3,2,0,0,0,3,0,2,D
4,201807,TRAIN_000004,13,0,1,5,4,0,1,1,...,1,1,0,0,0,0,0,0,0,E


In [29]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '대표결제일', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인트_마일리지_월적립_R3M', '포인트_포인트_월적립_B0M', '포인트_포인트_월적립_R3M', '포인트_적립포인트_R12M', '포인트_적립포인트_R3M', '포인트_이용포인트_R12M', '포인트_이용포인트_R3M', '포인트_잔여포인트_B0M', '마일_적립포인트_R12M', '마일_적립포인트_R3M', '마일_이용포인트_R12M', '마일_이용포인트_R3M', '마일_잔여포인트_B0M', '할인건수_R3M', '할인금액_R3M', '할인건수_B0M', '할인금액_B0M', '할인금액_청구서_R3M', '할인금액_청구서_B0M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '선결제건수_R6M', '선결제건수_R3M', '연체건수_R6M', '연체건수_R3M', '혜택수혜금액_R3M', '포인트_마일리지_환산_B0M', '혜택수혜금액', 'Segment']

문자형 컬럼 이름:
['ID', 'Segment']

숫자형 컬럼 이름:
['기준년월', '대표결제일', '대표결제방법코드', '대표청구지고객주소구분코드', '대표청구서수령지구분코드', '청구서수령방법', '청구서발송여부_B0', '청구서발송여부_R3M', '청구서발송여부_R6M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '포인트_마일리지_건별_B0M', '포인트_마일리지_건별_R3M', '포인트_포인트_건별_B0M', '포인트_포인트_건별_R3M', '포인트_마일리지_월적립_B0M', '포인

In [30]:
ex1 = ex1.fillna(-1)
ex1

,기준년월,ID,대표결제일,대표결제방법코드,대표청구지고객주소구분코드,대표청구서수령지구분코드,청구서수령방법,청구서발송여부_B0,청구서발송여부_R3M,청구서발송여부_R6M,...,상환개월수_결제일_R6M,상환개월수_결제일_R3M,선결제건수_R6M,선결제건수_R3M,연체건수_R6M,연체건수_R3M,혜택수혜금액_R3M,포인트_마일리지_환산_B0M,혜택수혜금액,Segment
0,201807,TRAIN_000000,27,0,0,2,2,1,1,1,...,5,3,0,0,1,0,3,0,0,D
1,201807,TRAIN_000001,13,0,1,5,4,1,1,1,...,6,3,0,0,0,0,0,0,0,E
2,201807,TRAIN_000002,1,0,0,6,5,1,1,1,...,6,3,0,0,0,0,121,0,50,C
3,201807,TRAIN_000003,5,0,1,5,4,1,1,1,...,5,3,2,0,0,0,3,0,2,D
4,201807,TRAIN_000004,13,0,1,5,4,0,1,1,...,1,1,0,0,0,0,0,0,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,25,0,1,5,4,0,0,0,...,0,0,0,0,0,0,0,0,0,E
2399996,201812,TRAIN_399996,20,0,0,6,5,1,1,1,...,6,3,0,0,0,0,164,0,53,D
2399997,201812,TRAIN_399997,20,0,2,5,4,1,1,1,...,6,3,0,0,0,0,0,0,0,C
2399998,201812,TRAIN_399998,20,0,1,5,4,0,0,0,...,0,0,0,0,0,0,0,0,0,E


In [31]:
num_df = ex1.select_dtypes(include=[np.number]).dropna()
low_var = num_df.var()[ num_df.var() < 0.01 ].index.tolist()
low_var

['대표결제방법코드']

In [32]:
cols_to_drop = ['대표결제방법코드']
ex1.drop(columns=cols_to_drop, inplace=True)

In [33]:
missing_mask = ex1.isna() | (ex1 == -1)
missing_ratio = missing_mask.mean()

high_na = missing_ratio[missing_ratio > 0.2].index.tolist()

high_const_cols = []
threshold_const = 0.8

for col in ex1.columns:
    top_ratio = ex1[col].value_counts(normalize=True, dropna=False).values[0]
    if top_ratio > threshold_const:
        high_const_cols.append(col)

to_drop = list(set(high_na + high_const_cols))

if 'Segment' in to_drop:
    to_drop.remove('Segment')

print("삭제 대상 컬럼 (결측>20% 또는 동일값>80%):", to_drop)

ex1.drop(columns=to_drop, inplace=True)


삭제 대상 컬럼 (결측>20% 또는 동일값>80%): ['포인트_잔여포인트_B0M', '포인트_포인트_월적립_B0M', '마일_이용포인트_R3M', '청구서발송여부_R6M', '포인트_마일리지_건별_R3M', '마일_적립포인트_R12M', '포인트_적립포인트_R3M', '포인트_마일리지_월적립_B0M', '할인건수_R3M', '포인트_마일리지_월적립_R3M', '포인트_포인트_월적립_R3M', '포인트_포인트_건별_R3M', '선결제건수_R6M', '연체건수_R3M', '포인트_마일리지_환산_B0M', '선결제건수_R3M', '마일_잔여포인트_B0M', '마일_이용포인트_R12M', '마일_적립포인트_R3M', '포인트_포인트_건별_B0M', '포인트_마일리지_건별_B0M', '할인건수_B0M', '연체건수_R6M']


In [34]:
import pandas as pd
import numpy as np

num_df = ex1.select_dtypes(include=[np.number]).dropna()

corr = num_df.corr().abs()

high_corr_pairs = (
    corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
        .stack()
        .reset_index()
)
high_corr_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']
high_corr_pairs = high_corr_pairs[high_corr_pairs['Correlation'] > 0.7]

if not np.issubdtype(ex1['Segment'].dtype, np.number):
    segment_map = {label: idx for idx, label in enumerate(sorted(ex1['Segment'].unique()))}
    ex1['Segment_encoded'] = ex1['Segment'].map(segment_map)
else:
    ex1['Segment_encoded'] = ex1['Segment']

segment_corr = ex1[num_df.columns].corrwith(ex1['Segment_encoded']).abs()

high_corr_pairs['Corr_with_Segment_1'] = high_corr_pairs['Feature_1'].map(segment_corr)
high_corr_pairs['Corr_with_Segment_2'] = high_corr_pairs['Feature_2'].map(segment_corr)

high_corr_pairs = high_corr_pairs.sort_values(by='Correlation', ascending=False).reset_index(drop=True)

print(f"▶ 상관계수 0.7 초과 변수쌍 수: {len(high_corr_pairs)}")
display(high_corr_pairs)


▶ 상관계수 0.7 초과 변수쌍 수: 21


,Feature_1,Feature_2,Correlation,Corr_with_Segment_1,Corr_with_Segment_2
0,대표청구서수령지구분코드,청구서수령방법,0.990924,0.018055,0.025964
1,할인금액_R3M,할인금액_B0M,0.969762,0.185652,0.180181
2,할인금액_청구서_R3M,할인금액_청구서_B0M,0.964109,0.202703,0.196683
3,혜택수혜금액_R3M,혜택수혜금액,0.956543,0.355754,0.350857
4,청구금액_B0,청구금액_R3M,0.954579,0.577988,0.590678
5,상환개월수_결제일_R6M,상환개월수_결제일_R3M,0.950930,0.149048,0.133415
6,청구금액_R3M,청구금액_R6M,0.942518,0.590678,0.597904
7,포인트_이용포인트_R12M,포인트_이용포인트_R3M,0.919594,0.143241,0.170152
8,청구금액_B0,청구금액_R6M,0.897863,0.577988,0.597904
9,할인금액_R3M,할인금액_청구서_R3M,0.859103,0.185652,0.202703


In [35]:
to_drop = []

for _, row in high_corr_pairs.iterrows():
    f1, f2 = row['Feature_1'], row['Feature_2']
    c1, c2 = row['Corr_with_Segment_1'], row['Corr_with_Segment_2']
    
    if pd.isna(c1) or pd.isna(c2):
        continue
    
    if c1 < c2:
        to_drop.append(f1)
    else:
        to_drop.append(f2)

to_drop = list(set(to_drop))

print(f"▶ 제거 대상 피처 수: {len(to_drop)}")
print("제거할 피처 목록:")
print(to_drop)

▶ 제거 대상 피처 수: 11
제거할 피처 목록:
['청구서발송여부_R3M', '청구금액_R3M', '혜택수혜금액', '할인금액_청구서_B0M', '할인금액_B0M', '할인금액_R3M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '대표청구서수령지구분코드', '포인트_이용포인트_R12M', '청구금액_B0']


In [36]:
cols_to_drop = ['청구서발송여부_R3M', '청구금액_R3M', '혜택수혜금액', '할인금액_청구서_B0M', '할인금액_B0M', '할인금액_R3M', '상환개월수_결제일_R6M', '상환개월수_결제일_R3M', '대표청구서수령지구분코드', '포인트_이용포인트_R12M', '청구금액_B0']
ex1.drop(columns=cols_to_drop, inplace=True)

In [37]:
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)


X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)


VIF > 10인 컬럼들: []
Empty DataFrame
Columns: [feature, VIF]
Index: []


In [38]:
ex1

,기준년월,ID,대표결제일,대표청구지고객주소구분코드,청구서수령방법,청구서발송여부_B0,청구금액_R6M,포인트_적립포인트_R12M,포인트_이용포인트_R3M,할인금액_청구서_R3M,혜택수혜금액_R3M,Segment,Segment_encoded
0,201807,TRAIN_000000,27,0,2,1,88693,3460,1296,0,3,D,3
1,201807,TRAIN_000001,13,1,4,1,16861,0,0,304,0,E,4
2,201807,TRAIN_000002,1,0,5,1,165221,17135,2765,0,121,C,2
3,201807,TRAIN_000003,5,1,4,1,127371,6464,2467,0,3,D,3
4,201807,TRAIN_000004,13,1,4,0,155,0,0,0,0,E,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,25,1,4,0,0,2379,2049,0,0,E,4
2399996,201812,TRAIN_399996,20,0,5,1,99849,64696,13202,162,164,D,3
2399997,201812,TRAIN_399997,20,2,4,1,41073,0,0,0,0,C,2
2399998,201812,TRAIN_399998,20,1,4,0,0,0,0,0,0,E,4


In [39]:
from sklearn.preprocessing import LabelEncoder

df_corr = ex1.drop(columns=['기준년월', 'ID']).copy()

le = LabelEncoder()
df_corr['Segment_encoded'] = le.fit_transform(df_corr['Segment'])

correlations = df_corr.corr(numeric_only=True)['Segment_encoded'].drop('Segment_encoded')

selected_cols = correlations[correlations.abs() > 0.1].index.tolist()

final_cols = ['기준년월', 'ID','Segment'] + selected_cols

ex1_filtered = ex1[final_cols]


In [40]:
cols = ex1_filtered.columns.tolist()
cols

['기준년월',
 'ID',
 'Segment',
 '청구서발송여부_B0',
 '청구금액_R6M',
 '포인트_적립포인트_R12M',
 '포인트_이용포인트_R3M',
 '할인금액_청구서_R3M',
 '혜택수혜금액_R3M']

In [41]:
ex1_filtered.to_parquet('청구_전처리_Segment.parquet', index=False)